Packages

In [8]:
import os
import numpy
import pandas as pd
import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/01 10:15:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
sdf = read_file(spark, 'data', '../data/curated/')

|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------------------------
 user_id         | 7                                    
 merchant_abn    | 71041015148                          
 dollar_value    | 226.70111326425848                   
 order_id        | ba1c3c0b-6143-49c0-8bb5-c361dabd91a2 
 merchant_name   | Orci Ltd                             
 tags            | digital goods: books, movies, music  
 take_rate       | 2.19                                 
 type            | c                                    
 postcode        | 4606                                 
 consumer_id     | 511685                               
 name            | Andrea Jones                         
 address         | 122 Brandon Cliff                    
 state           | QLD                                  
 gender          | Female                               
 POA_CODE21      | 4606                                 
 2020_popu       | 552.6292160121676                    
 2021_popu

In [13]:
# type
type_indexer = StringIndexer(inputCol = "type", outputCol = "typeIndex", handleInvalid="skip")

# gender
gender_indexer = StringIndexer(inputCol = "gender", outputCol = "genderIndex", handleInvalid="skip")

# tag
tag_indexer = StringIndexer(inputCol = "tags",outputCol = "tagIndex", handleInvalid="skip")
onehotencoder_tag_vector = OneHotEncoder(inputCol = "tagIndex", outputCol = "tag_vec")

#Create pipeline and pass all stages
pipeline = Pipeline(stages=[gender_indexer, 
                            type_indexer,
                            tag_indexer,
                            onehotencoder_tag_vector])

In [14]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-----+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|New cases / day|     month|total|order_datetime|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-----+--------------+
|      7| 71041015148|226.70111326425848|ba1c3c0b-6143-49c...|            Orci Ltd|digital goods: bo...|     2.19|   c|    4606|     511

In [15]:
sdf_transformed = pipeline.fit(sdf).transform(sdf)
sdf_transformed.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|New cases / day|     month|total|order_datetime|genderIndex|typeIndex|tagIndex|       tag_vec|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+--------------

- figure out that tag_vec

In [16]:
SAMPLE_SIZE = 0.05
sdf_transformed = sdf_transformed.sample(SAMPLE_SIZE, seed=0)

In [17]:
df_transformed = sdf_transformed.toPandas()
df_transformed.head()

,user_id,merchant_abn,dollar_value,order_id,merchant_name,tags,take_rate,type,postcode,consumer_id,...,2020_popu,2021_popu,New cases / day,month,total,order_datetime,genderIndex,typeIndex,tagIndex,tag_vec
0,26,43186523025,48.785919,38a3e0bd-d40d-494e-8265-56972e2651f8,Lorem Ipsum Sodales Industries,"florists supplies, nursery stock, and flowers",4.47,b,3469,1421465,...,488.8355158678476,495.0634561440711,"1,621",2021-11-01,33345,2021-11-26,1.0,1.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,77,21587176464,116.839682,9a7ac254-08fc-4323-af11-234bfb513937,Dictum Eu LLP,motor vehicle supplies and new parts,3.37,b,6603,577363,...,592.9802020302061,587.7232853477678,"1,621",2021-11-01,33345,2021-11-26,0.0,1.0,15.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,167,41881349154,172.441693,fe46900a-d05c-4397-a033-df6ec00481ac,Adipiscing Enim LLP,"books, periodicals, and newspapers",2.02,c,5491,356749,...,1166.040395168675,1173.042138907628,"1,621",2021-11-01,33345,2021-11-26,1.0,2.0,12.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,241,23692710924,187.171168,7b183c0f-e530-428a-95df-ce470c075358,Integer Urna Corporation,"florists supplies, nursery stock, and flowers",5.61,a,4571,1273702,...,5957.860236782877,6044.134588593148,"1,621",2021-11-01,33345,2021-11-26,1.0,0.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,296,57900494384,37.700793,5952b071-c769-49b0-9992-1e9ec14f4b6e,Porttitor Tellus Corporation,tent and awning shops,6.39,a,3744,1241110,...,90.05519496772257,91.37710608651483,"1,621",2021-11-01,33345,2021-11-26,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


- Drop unused column

In [18]:
sdf_transformed = sdf_transformed.drop('order_id', 'merchant_name', 'tags', 'type', 'name', 'address', 'state', 'gender')

In [19]:
sdf_transformed.show()

+-------+------------+------------------+---------+--------+-----------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+---------------+
|user_id|merchant_abn|      dollar_value|take_rate|postcode|consumer_id|POA_CODE21|        2020_popu|        2021_popu|New cases / day|     month|total|order_datetime|genderIndex|typeIndex|tagIndex|        tag_vec|
+-------+------------+------------------+---------+--------+-----------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+---------------+
|     26| 43186523025| 48.78591885045311|     4.47|    3469|    1421465|      3469|488.8355158678476|495.0634561440711|          1,621|2021-11-01|33345|    2021-11-26|        1.0|      1.0|     5.0| (24,[5],[1.0])|
|     77| 21587176464|116.83968187184298|     3.37|    6603|     577363|      6603|592.9802020302061|587.7232853477678|          1,621|2021-

In [20]:
from pyspark.sql.functions import year, month, dayofmonth

In [25]:
sdf_transformed_year = sdf_transformed.withColumn("order_year", year('order_datetime'))

In [37]:
sdf_transformed_year.show()

+-------+------------+------------------+---------+--------+-----------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+---------------+----------+
|user_id|merchant_abn|      dollar_value|take_rate|postcode|consumer_id|POA_CODE21|        2020_popu|        2021_popu|New cases / day|     month|total|order_datetime|genderIndex|typeIndex|tagIndex|        tag_vec|order_year|
+-------+------------+------------------+---------+--------+-----------+----------+-----------------+-----------------+---------------+----------+-----+--------------+-----------+---------+--------+---------------+----------+
|     26| 43186523025| 48.78591885045311|     4.47|    3469|    1421465|      3469|488.8355158678476|495.0634561440711|          1,621|2021-11-01|33345|    2021-11-26|        1.0|      1.0|     5.0| (24,[5],[1.0])|      2021|
|     77| 21587176464|116.83968187184298|     3.37|    6603|     577363|      6603|592.980202030

In [40]:
sdf_transformed_year = sdf_transformed_year.drop('2020_popu')